In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from PIL import Image
import os

C:\Users\chao_gu\Anaconda3\envs\tensorflow_workshop\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## load pictures

In [2]:
IMAGE_SHAPE = (200,300)

In [3]:
IMAGE_SHAPE_INPUT = (200,300,3)

In [4]:
path = os.getcwd() + '\\house_image\\'

In [5]:
folders = os.listdir(path)

In [6]:
bedrooms = [path+i+'\\'+'bedroom.jpg' for i in folders]
bathrooms = [path+i+'\\'+'bathroom.jpg' for i in folders]
living_rooms = [path+i+'\\'+'living_room.jpg' for i in folders]
kitchens = [path+i+'\\'+'kitchen.jpg' for i in folders]

## build model - VGG

In [7]:
#weight to normalize image pixel value for vgg19 model
RGB_MEAN_PIXELS = np.array([123.68,116.779,103.939]).reshape((1,1,1,3)).astype(np.float32)

In [8]:
import keras

Using TensorFlow backend.


In [9]:
def get_feature_matrix_vgg(path_list):
    image_array = []
    for img in path_list:
        im = Image.open(img)
        im = im.resize(IMAGE_SHAPE[::-1])
        image_array.append(np.array(im))
    image_array = np.array(image_array)
    vgg19 = keras.applications.VGG19(weights='imagenet', include_top=False)
    embedding = []
    for i in image_array:
        img_input = np.expand_dims(i,0)
        keras_output = vgg19.predict(np.flip(img_input-RGB_MEAN_PIXELS,axis = -1))
        embedding.append(keras_output)
    return np.array(embedding)

In [11]:
bedrooms_embedded_vgg = get_feature_matrix_vgg(bedrooms)
bathrooms_embedded_vgg = get_feature_matrix_vgg(bathrooms)
kitchens_embedded_vgg = get_feature_matrix_vgg(kitchens)
living_rooms_embedded_vgg = get_feature_matrix_vgg(living_rooms)

In [12]:
bedrooms_embedded_vgg.shape

(125, 1, 6, 9, 512)

In [13]:
def similarity_matrix(image_embedded):
    n = image_embedded.shape[0]
    similarity_mat = np.zeros((n,n))
    for i in range(n):
        img_i = image_embedded[i].flatten()
        for j in range(n):
            img_j = image_embedded[j].flatten()
            result = 1-spatial.distance.cosine(img_i,img_j)
            similarity_mat[i][j] = result
    return similarity_mat

In [14]:
from scipy import spatial

In [15]:
bedrooms_sim = similarity_matrix(bedrooms_embedded_vgg)
bathrooms_sim = similarity_matrix(bathrooms_embedded_vgg)
kitchens_sim = similarity_matrix(kitchens_embedded_vgg)
living_rooms_sim = similarity_matrix(living_rooms_embedded_vgg)

In [16]:
import pickle

In [17]:
with open('bedroom_sim_mat.pickle','wb') as f:
    pickle.dump(bedrooms_sim,f)

In [18]:
with open('bathroom_sim_mat.pickle','wb') as f:
    pickle.dump(bathrooms_sim,f)

In [19]:
with open('kitchen_sim_mat.pickle','wb') as f:
    pickle.dump(kitchens_sim,f)

In [20]:
with open('living_room_sim_mat.pickle','wb') as f:
    pickle.dump(living_rooms_sim,f)